In [3]:
!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
     -------------------------------------- 344.4/344.4 kB 7.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
####한국어 형태소 분석기####
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
okt = Okt()

df_18=pd.read_csv('../원본데이터/news18.csv',encoding='cp949')
df_19=pd.read_csv('../원본데이터/news19.csv',encoding='cp949')
df_20=pd.read_csv('../원본데이터/news20.csv',encoding='cp949')
df_21=pd.read_csv('../원본데이터/news21.csv',encoding='cp949')

df_all=pd.concat([df_18,df_19,df_20,df_21])

df_all

df_all['time']=pd.to_datetime(df_all['time'])
df_all['datetime']=df_all['time'].dt.strftime('%Y-%m-%d')

df_all['datetime']=pd.to_datetime(df_all['datetime'])

df_new=df_all

df_new.reset_index(inplace=True)

df_new=df_new.groupby(['datetime'], as_index=False).agg(lambda x: ' '.join(set(x)))

df_new.head()

df_new

df_new.tail(1000)

week = [g for n, g in df_new.set_index('datetime').groupby(pd.Grouper(freq='W'))]

week[40]

C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)
C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,theme,title,article
datetime,,,
2018-10-08,오늘의 뉴스,"한돈, 원료육 비중 갈수록 높아져 한돈업계, 10월 한돈 소비 진작 위해 팔 걷어",돼지고기 시장에서 원료육(햄 소시지 등 가공품)의 비중이 날로 커지고 있는 것으로 ...
2018-10-10,오늘의 뉴스,양돈장 성적 수년간 답보 상태 PED 유행 시작되나…제주 주의보,올해도 양돈장 평균 생산 성적이 제자리걸음하고 있다. 2년 연속으로 MSY가 18두...
2018-10-11,오늘의 뉴스,양돈장 직수입 종돈 크게 늘어 식육 가공품 수입 증가세 ‘광폭’ ASF 위기 넘기면...,국내 식육 가공품 시장의 성장과 함께 식육 가공품 수입량도 매년 빠르게 늘고 있다....


### txt로 저장

In [6]:
error_index=[]
stem_list=[]

for index, text in enumerate(week):
    try:
        stem_list.append('{}번쨰 기사입니다.'.format(index))
        stem_list.append(text['article'][0])
    except:
        stem_list.append('NULL')
        error_index.append(index)
    with open('./news_stem/stem_list{}.txt'.format(index),'w',encoding='UTF-8') as f:
        f.write('\n'.join(map(str, stem_list)))

In [47]:
import json
import sys
import requests

file = open('./positive_negative_text_result2.txt', 'w', encoding='utf8') #결과파일
file2=open('./result.txt', 'w', encoding='utf8') #결과파일
n = 1 #첫번째
m = 800  #두번째
for i in range(90, 100) : #41번
    client_id = "4q7skv8ste"  #클라이언트 키
    client_secret = "wmqQzlBqk2InZAdvDsHOHDZNIVO9PVBysFVxy8m4" #클라이언트 시크릿 키

    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }
#     for j in range(len(week)):
#         print("{}번째 기사입니다.".format(j)+"\n"+"\n"+"\n")
    f = open('./news_stem/stem_list207.txt', 'r',encoding='UTF-8')

    text = f.read()
    text_2 = text.replace("\n", "")
    text_3 = text_2.replace("\t", "")
    text_4 = text_3.replace("________________", "0")
    content = text_4[n:m] #800단어씩
    print(content)
    print("\n")

    data = {
      "content": content
    }
    print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        print (response.text)
    else:
        print("Error : " + response.text)

    f.close()    

    text = response.json()

    raw = text['sentences']

    file2.write(str(raw) + '\n'+"\n")
    max_cnt = len(raw) #리스트 개수

    for i in range(0, max_cnt) : 
        content = str(raw[i]['content']) #텍스트
        offset = int(raw[i]['highlights'][0]['offset']) #오프셋
        length = int(raw[i]['highlights'][0]['length']) #해당 길이
        text = content[offset : offset + length] #부정 의미가 있는 문구 텍스트 추출
        sentiment=str(raw[i]['sentiment'])
        #print(text)
        file.write(str(text)+"\t"+str(sentiment) + '\n')

    n += 800
    m += 800

file.close()

번쨰 기사입니다.17년에도 한돈 5000원 못 이뤄4,958원 전년보다 7.7% ↑10월 이후 약세로 역부족지난해 돼지 값이 전년 대비 오르면서 4년 연속 4천500원(kg당 탕박) 이상을 기록했다. 지난해 평균 돼지 값은 4천958원(12월 27일 기준)으로 지난 16년 4천600원보다 7.7% 가량 올랐다. 이로써 지난 14년 이후 돼지 값은 연평균 4천500원 이상을 기록하며 호황을 이어갔다. 돼지 출하두수 증가에도 소비가 늘면서 돈가 강세를 뒷받침한 결과다. 그러나 평균 5천원의 문턱은 넘지 못했다. 지난 9월까지 전년 대비 상승세를 기록해왔던 돼지 값이 10월 이후 물량 증가로 강세를 이어가지 못했기 때문이다.  농가 55% MSY 18두 미만한돈 하락 시 경영 악화 우려이유 후 육성에 관심 집중을양돈농가 중 절반이 넘는 농가가 MSY 18두 미만을 기록하고 있는 것으로 나타났다.한돈협회는 최근 2017년도 한돈팜스 전산 성적 보고회를 개최, 지난해 1~10월까지의 한돈팜스 사용 평균 3천320농가의 생산 성적을 최종 취합한 결과, 평균 MSY는 17.7두 분석했다. 이 가운데 조사 농가의 54.8%가 MSY 평균 18두에도 못 미치는 17두 이하를 기록, 한돈가격 하락시 이들 농가들의 경영악화가 우려되고 있다.MSY별 농가수 비율 분석 자료에 따르면 3천320농가 중 △16두 미만 농가=41% △16~17두=17% △18~19두=14% △20~21두=11% △22~23두=7% △24~25두=5% △26두 이상=6%로 나타났다. 특히 2016년과 비교해 봤을 때 MSY 18두 미만 비율은 56.5%


{
    "content": "\ubc88\uca30 \uae30\uc0ac\uc785\ub2c8\ub2e4.17\ub144\uc5d0\ub3c4 \ud55c\ub3c8 5000\uc6d0 \ubabb \uc774\ub9044,958\uc6d0 \uc804\ub144\ubcf4\ub2e4 7.7% \u219110\uc6d4 \uc774\ud6c4 \u

{"document":{"sentiment":"positive","confidence":{"negative":14.64305,"positive":73.59707,"neutral":11.759879}},"sentences":[{"content":"서 57.2%로 증가됐고, 24두 이상을 보인 농가 역시 10.5%에서 10.9%로 소폭 늘어났다.","offset":0,"length":55,"sentiment":"neutral","confidence":{"negative":1.806754E-4,"positive":4.7098863E-4,"neutral":0.9993482},"highlights":[{"offset":47,"length":7}]},{"content":"이 같이 전체적으로 국내 양돈장 생산성이 저하되고 있는 요인은 질병 상재화 및 폭염으로 인한 관리가 미흡, 이유 후 육성률이 낮아진 영향으로 풀이되고 있다.","offset":55,"length":87,"sentiment":"negative","confidence":{"negative":0.998044,"positive":3.9667526E-4,"neutral":0.0015593478},"highlights":[{"offset":60,"length":24}]},{"content":"  작년 돼지 출하 사상최고 기록1670만두, 전년비 1% 늘어한돈 호조로 입식 는 영향새해 역시 증가세 이어갈 듯돼지 출하두수가 2년 연속 사상 최고치를 경신했다.","offset":142,"length":92,"sentiment":"positive","confidence":{"negative":0.0014036603,"positive":0.99791676,"neutral":6.7963015E-4},"highlights":[{"offset":36,"length":25}]},{"content":" 지난해 12월 돼지 출하물량은 27일 현재 130만마리로 11월까지 출하된 돼지 1천526만마리까지 더하면 1천656만마리로 이미 

{"document":{"sentiment":"positive","confidence":{"negative":0.019656844,"positive":57.120983,"neutral":42.85936}},"sentences":[{"content":" 종돈 4,409마리 수입…5년만에 최고종개협  신규 종돈장 입식에다 일반농장 직수입 여파작년 종돈 수입 물량이 크게 증가하며 5년 만에 4천두를 넘어섰다.","offset":0,"length":87,"sentiment":"neutral","confidence":{"negative":2.1090436E-4,"positive":0.005187769,"neutral":0.99460137},"highlights":[{"offset":20,"length":66}]},{"content":"종축개량협회에 따르면 지난해 종돈수입두수는 통관 기준 4천409두로 전년 3천76두에 비해 43.3% 크게 증가했다.","offset":87,"length":65,"sentiment":"neutral","confidence":{"negative":6.96E-4,"positive":0.001003202,"neutral":0.99830085},"highlights":[{"offset":12,"length":43}]},{"content":" 이는 FMD 이후 △11년=7천171두 △12년=4천744두에 이어 사상 3번째 많은 물량이다.","offset":152,"length":54,"sentiment":"neutral","confidence":{"negative":5.5671774E-4,"positive":8.663004E-4,"neutral":0.998577},"highlights":[{"offset":33,"length":20}]},{"content":"국가별로 보면 캐나다가 2천347두로 가장 많았으며 그 다음이 프랑스 1천258두, 덴마크 574두, 미국 230두 순으로 집계됐다.","offset":206,"length":74,"sent

{"document":{"sentiment":"negative","confidence":{"negative":64.49438,"positive":16.226091,"neutral":19.279531}},"sentences":[{"content":"7.2마리로 전년도 17.3두에 비해 낮았다.","offset":0,"length":25,"sentiment":"neutral","confidence":{"negative":6.052675E-4,"positive":2.01723E-4,"neutral":0.999193},"highlights":[{"offset":15,"length":9}]},{"content":" 16년 모돈 두수가 94만4천마리로 일년전보다 1.6%, 돼지 도축두수는 1.1% 증가했다.","offset":25,"length":52,"sentiment":"neutral","confidence":{"negative":2.1938358E-4,"positive":9.3831343E-4,"neutral":0.9988424},"highlights":[{"offset":33,"length":12}]},{"content":" 특히 16년에 비해 상대적으로 더위 피해가 크지 않았음에도 생산성은 뒷걸음질 친 것이다.","offset":77,"length":50,"sentiment":"negative","confidence":{"negative":0.9959084,"positive":0.0011084195,"neutral":0.0029831822},"highlights":[{"offset":34,"length":15}]},{"content":" 다산성 모돈의 도입은 갈수록 증가하고 있는 가운데 자돈 질병 피해가 여전히 큰 때문으로 풀이되고 있다.","offset":127,"length":58,"sentiment":"negative","confidence":{"negative":0.99810505,"positive":3.5366154E-4,"neutral":0.00154

{"document":{"sentiment":"negative","confidence":{"negative":78.28508,"positive":13.998995,"neutral":7.715927}},"sentences":[{"content":"고돈가 시대…생산성 제고 시급새해 출하 1700만두 넘어…사상 최고공급량 증가 속 호재 없어 약세 불가피이유 후 육성률 높여 경영 악화 줄여야양돈 PM 설문올해 돼지 출하는 1천700만두를 넘는 사상 최고치가 예상되는 가운데 호황을 누렸던 지난 4년에 견줘 돼지 값은 약세가 불가피할 것으로 전망됐다.","offset":0,"length":168,"sentiment":"negative","confidence":{"negative":0.9985727,"positive":7.776494E-4,"neutral":6.4953975E-4},"highlights":[{"offset":144,"length":23}]},{"content":"양돈타임스가 사료회사 PM 6명을 대상으로 올 한해 양돈 시장 전망을 들어본 결과, 구체적인 수치에서는 차이가 있었지만 공통적으로 돼지 출하 증가와 이에 따른 돼지 값 하락을 점쳤다.","offset":168,"length":102,"sentiment":"negative","confidence":{"negative":0.9984779,"positive":3.246395E-4,"neutral":0.0011975925},"highlights":[{"offset":89,"length":12}]},{"content":" 지난해 모돈 사육두수가 100만두를 넘기면서 올해 돼지 사육두수는 1천100만두 안팎으로 늘고 자연히 출하물량 역시 1천700만마리는 무난히 넘길 것이란 전망이다.","offset":270,"length":92,"sentiment":"neutral","confidence":{"negative":4.5624937E-4,"positive":0.0046940823,"neutral":0.99484

{"document":{"sentiment":"positive","confidence":{"negative":31.476946,"positive":62.002747,"neutral":6.5203085}},"sentences":[{"content":"천65억원서 2배 가량 늘었으며 농가당 매출도 8억4천만원서 14억원으로 64% 증가했다.","offset":0,"length":50,"sentiment":"positive","confidence":{"negative":6.9094566E-4,"positive":0.9984718,"neutral":8.372895E-4},"highlights":[{"offset":45,"length":4}]},{"content":"그런데 이번 조치로 제주산에 비해 상대적으로 저렴한 타 지역 돼지고기가 유입되면서 도내 가격과 전국 가격 간 격차가 줄어들 것이란 전망이다.","offset":50,"length":78,"sentiment":"positive","confidence":{"negative":0.004838572,"positive":0.993486,"neutral":0.0016754159},"highlights":[{"offset":19,"length":58}]},{"content":" 실제 지난달 13일 기준 제주 돼지 값은 ㎏당 4천805원으로 반입 직전일인 10월 12일 5천887원에 비해 18.3% 하락했으며 이로 인해 타 지방산 한돈과의 가격 격차가 1천324원서 335원으로 줄었다고 지적했다.","offset":128,"length":124,"sentiment":"negative","confidence":{"negative":0.9920677,"positive":4.1563087E-4,"neutral":0.007516721},"highlights":[{"offset":104,"length":19}]},{"content":"한국은행은 현재까지 제주로 반입된 다른 지역 한돈물량이 도내 도축물량의 1% 미만으로 그 영향이 

In [48]:
raw

[{'content': '천65억원서 2배 가량 늘었으며 농가당 매출도 8억4천만원서 14억원으로 64% 증가했다.',
  'offset': 0,
  'length': 50,
  'sentiment': 'positive',
  'confidence': {'negative': 0.00069094566,
   'positive': 0.9984718,
   'neutral': 0.0008372895},
  'highlights': [{'offset': 45, 'length': 4}]},
 {'content': '그런데 이번 조치로 제주산에 비해 상대적으로 저렴한 타 지역 돼지고기가 유입되면서 도내 가격과 전국 가격 간 격차가 줄어들 것이란 전망이다.',
  'offset': 50,
  'length': 78,
  'sentiment': 'positive',
  'confidence': {'negative': 0.004838572,
   'positive': 0.993486,
   'neutral': 0.0016754159},
  'highlights': [{'offset': 19, 'length': 58}]},
 {'content': ' 실제 지난달 13일 기준 제주 돼지 값은 ㎏당 4천805원으로 반입 직전일인 10월 12일 5천887원에 비해 18.3% 하락했으며 이로 인해 타 지방산 한돈과의 가격 격차가 1천324원서 335원으로 줄었다고 지적했다.',
  'offset': 128,
  'length': 124,
  'sentiment': 'negative',
  'confidence': {'negative': 0.9920677,
   'positive': 0.00041563087,
   'neutral': 0.007516721},
  'highlights': [{'offset': 104, 'length': 19}]},
 {'content': '한국은행은 현재까지 제주로 반입된 다른 지역 한돈물량이 도내 도축물량의 1% 미만으로 그 영향이 미미한 상황이지만 향

In [6]:
week[207]['article'][2]

'2011년 구제역 사태 이후 10년이 흘렀다. 당시 300만마리 넘는 돼지가 살처분 되는 등 방역 대란이 일어났다. 이에 정부는 가축 질병 방역 체계 선진화라는 명목으로 방역 관련 기관 및 인원을 크게 늘렸다. 그러나 방역 관련 ‘뱃사공’이 많아지면서 농가 규제만 많아지고, 방역 효율성은 저하되면서 방역 정책은 ‘산’으로 가고 있다는 지적이 일고 있다.지난 2011년 당시 농림수산식품부는 구제역 발생을 계기로 방역시스템을 획기적으로 개편하고, 축산업의 체질을 근본적으로 바꾸어 나가기 위해 ‘가축질병 방역체계 개선 및 축산업 선진화 방안’을 발표했다. 이를 통해 앞으로 10년을 내다보고 국민에게 사랑받는 선진 축산업, 친환경 축산업을 목표로 삼았던 것이다.이에 따라 10년간 정부는 농가 축산업 허가제 도입 및 가축질병 방역을 위한 조직도 정비하고 각종 방역 산하 기관을 설립했다. 우선 국립수의과학검역원 등 3개 분야를 농림축산검역본부로 통합하고, 전국 지자체 10개도 가축질병방역세터, 백신전문연구센터 설립, 동물방역통합시스템(KAHIS) 구축했다. 특히 2017년 방역정책과를 국(局)으로 확대, 2년 후 정규화했다. 이어 축산환경관리원, 야생동물질병관리원 등 환경과 질병 예방을 위한 산하기관도 설립됐다.이 같이 질병 방역을 위한 농축산부 내 방역국 설치 및 산하기관 설립으로 방역 정책은 보다 효율적이고, 신속하게 대응할 것으로 예측됐으나 농가 피해를 되레 줄지 않고 있다는 여론이 높다. 특히 ASF?AI 등으로 인한 무분별한 살처분 정책으로 농가는 재산권 상실, 소비자는 해당 육류 소비자 값 상승 등 모두 피해를 겪고 있다. 아울러 방역정책국을 중심으로 농가 책임 강화 및 과태료 등 농가 규제는 더욱 양산하면서 농가들의 신뢰를 잃고 있다.이에 농가 및 현장 전문가들은 뱃사공이 많으면 산으로 갈 수 밖에 없다며 정부 방역 조직의 효율적으로 이용할 수 있도록, 현장 의견 수렴 및 현장 전문가들의 의견을 보다 더 반영할 수 있도록 해야 한다고 주장하고 있다. 특히 

In [46]:
raw

[{'content': '번쨰 기사입니다.',
  'offset': 0,
  'length': 9,
  'sentiment': 'neutral',
  'confidence': {'negative': 0.0010830899,
   'positive': 0.0003374724,
   'neutral': 0.9985794},
  'highlights': [{'offset': 0, 'length': 8}]},
 {'content': '17년에도 한돈 5000원 못 이뤄4,958원 전년보다 7.7% ↑10월 이후 약세로 역부족지난해 돼지 값이 전년 대비 오르면서 4년 연속 4천500원(kg당 탕박) 이상을 기록했다.',
  'offset': 9,
  'length': 102,
  'sentiment': 'negative',
  'confidence': {'negative': 0.9979988,
   'positive': 0.00046002472,
   'neutral': 0.0015411132},
  'highlights': [{'offset': 41, 'length': 43}]},
 {'content': ' 지난해 평균 돼지 값은 4천958원(12월 27일 기준)으로 지난 16년 4천600원보다 7.7% 가량 올랐다.',
  'offset': 111,
  'length': 63,
  'sentiment': 'neutral',
  'confidence': {'negative': 0.00026012835,
   'positive': 0.0009816206,
   'neutral': 0.9987582},
  'highlights': [{'offset': 56, 'length': 6}]},
 {'content': ' 이로써 지난 14년 이후 돼지 값은 연평균 4천500원 이상을 기록하며 호황을 이어갔다.',
  'offset': 174,
  'length': 50,
  'sentiment': 'positive',
  'confidence': {'negative': 0.000

In [49]:
df_new = pd.DataFrame(raw)

In [55]:
df_new.iloc[9]

content       MSY의 경우 두당 섭취량 증가와 출하일령 감소로 19.1두를 기록, 연중 가장 높...
offset                                                      627
length                                                       72
sentiment                                               neutral
confidence    {'negative': 0.2541937, 'positive': 0.06502039...
highlights                        [{'offset': 5, 'length': 32}]
Name: 9, dtype: object

In [7]:
df_new

,datetime,theme,title,article
0,2018-01-04,오늘의 뉴스,"종돈 4,409마리 수입…5년만에 최고(1/4) 2018년, 축산 사육 환경 변화 ...","\t\t\t\t\t2018년, 축산 사육 환경 변화 원년\r\n김 장관 신년사서 밝..."
1,2018-01-11,오늘의 뉴스,제주도 양돈농가 채산성 악화 전망(1/11) 후보돈 판매 20만두 넘었다(1/11)...,\t\t\t\t\t저무는 고돈가 시대…생산성 제고 시급\r\n새해 출하 1700만두...
2,2018-01-18,오늘의 뉴스,냉장 삼겹 관세 인하(1/18) 약세 한돈 중순 이후 ‘안정’(1/18) 한돈 품질...,\t\t\t\t\t약세 한돈 중순 이후 ‘안정’\r\n출하 줄고 올림픽 수요 기대\...
3,2018-01-25,오늘의 뉴스,탕박 등급제 시행 고작 20%(1/25) ‘안전 한돈’ 생산에 정부-농가 ‘엇박자’...,\t\t\t\t\t새해 자조금 예산 343억원 확정\r\n농축산부 승인…홍보비 1위...
4,2018-02-01,오늘의 뉴스,한돈 햄·소시지 比 수출 길 열려(2/1) [양돈전망]한돈 올해 내년 하락하다 20...,\t\t\t\t\tPED 극성…농장 피해 속출\r\n12월 기점서 전국으로 확산\r...
...,...,...,...,...
583,2021-12-14,오늘의 뉴스,연내 축산 온실가스 감축 전략 나온다 8대 방역시설 의무화에 또 긴장 고조 양돈사료...,8대 방역시설 모식도정부가 전국 양돈농장에 8대 방역시설 의무화를 위한 법령 개정을...
584,2021-12-15,오늘의 뉴스,"과태료는 ‘대폭’ 인센티브는 ‘찔끔’ 출하도 늘었는데 12월 강세, 그 이유는?",정부가 ASF 질병 방역 강화를 위한 8대 방역시설 설치 의무화를 추진하고 있는 가...
585,2021-12-21,오늘의 뉴스,CPTPP 가입 공식화…농축산업 “가입 철회 하라”,정부가 ‘포괄적 점진적 환태평양경제동반자협정(CPTPP)’ 가입 추진을 공식화하면서...
586,2021-12-22,오늘의 뉴스,생석회 도포 안하면 과태료 축산 미생물제 효과 공공차원서 검증,냄새 저감 등에 효과가 있다고 알려진 축산 미생물제에 대한 공공 차원의 실증 사업이...
